# Onward to AWS!
- Now that you saw how long those CPUs can take...
- Let's go to GPUs!
- That means AWS...

## Setting up a Deep Learning AWS GPU Instance
- You'll typically want to use a **Deep Learning AMI** that comes with all the relevant packages pre-installed and nice default configurations.
- Now follow along with me as we venture Beyond the Wall!
- **Caution**: This will cost $.  If you don't have your AWS credits, be wary.

## Launching the Instance
1. Proceed to the AWS EC2 Console
2. Click "Launch Instance"
3. Click Community AMIs
4. Search for / find the Deep Learning AMI
5. Select it
6. Select a GPU instance (e.g. g2.2xlarge) and click "Next: Configure Instance Details" - the rest is the usual AWS steps!

## Configuration
1. Log onto your server via:
  - `ssh -i <your_keypair.pem> icarus@<your_public_ip>`
  - Password: `changetheworld`
2. Log out
3. In EC2 Console, reboot your instance (for init scripts to run)
4. Log back in again (#1 above) and let's go!

## Start a Jupyter Notebook (on your server)
- You know how to do this!:
  - `jupyter notebook`
- If you want it to be open to the internet, you'll need to follow these instructions:
  - [Running Jupyter Notebook Server in AWS](http://jupyter-notebook.readthedocs.io/en/latest/public_server.html)
- In your local browser, browse to:
  - `https://<your_public_ip>:8888`
  - Note, you'll probably have to `pip install gensim` on the server
- And let's begin!

# AWS LSTM + CNN Example: Sentiment 1 more time!
- Let's marvel at how blazing fast the same example runs on your GPU instance!

In [ ]:
import sys
import random
import numpy as np


from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import (Embedding, Convolution1D, Dropout, MaxPooling1D, 
                          LSTM, Dense, Activation)
from keras.optimizers import RMSprop


# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
filter_length = 5
nb_filter = 64
pool_length = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
nb_epoch = 2

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')

aws_cnn_model = Sequential()
aws_cnn_model.add(Embedding(max_features, embedding_size, input_length=maxlen))
aws_cnn_model.add(Dropout(0.25))
aws_cnn_model.add(Convolution1D(filters=nb_filter,
                                kernel_size=filter_length,
                                padding='valid',
                                activation='relu',
                                strides=1))
aws_cnn_model.add(MaxPooling1D(pool_size=pool_length))
aws_cnn_model.add(LSTM(lstm_output_size))
aws_cnn_model.add(Dense(1))
aws_cnn_model.add(Activation('sigmoid'))

aws_cnn_model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

print('Train...')
filname = "models/aws_cnn_model.pkl"
aws_cnn_model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
                  validation_data=(X_test, y_test))
score, acc = aws_cnn_model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)